# Investigation: User Creation with Administrator Policy Assigned

This notebook assists in investigating this detection.

In [ ]:
from scanner_client import ScannerClient
import pandas as pd
import plotly.express as px

# Initialize Scanner Client
client = ScannerClient(api_url="https://api.kast-apse1.scanner.dev", api_key="YOUR_API_KEY")

In [ ]:
query = """
%ingest.source_type:aws:cloudtrail
(eventName:"CreateUser" OR (eventName:"AttachUserPolicy" AND requestParameters:"AdministratorAccess"))
| parse_json requestParameters "userName" as rp_userName
| parse_json responseElements "user"
| parse_json user "userName" as re_userName
| eval target_user = coalesce(rp_userName, re_userName)
| stats count_distinct(eventName) as unique_actions by target_user
| where unique_actions == 2
"""

results = client.query(query, time_range_s=86400)
df = pd.DataFrame(results)
df.head()